# 操作手順
---

- L4 GPUでインスタンスを起動する
- 全てのセルを実行する
- 最後の「サーバ起動用セル」に出力される`PUBLIC_URL`を取得しておく
    - このURLはセル起動ごとに変わるユニークなURLである点に注意
    - このURLがAPIエンドポイントのベースとなる
- 例えばPOSTMANでエンドポイントにリクエストを投げる

```
    - POST https://{unique-ids}.ngrok-free.app/predict
    - Body: raw.JSON
            {
                "message": "ここに何か文字列を入力する"
            }
    
    → Send!
```

In [1]:
!pip install transformers accelerate bitsandbytes fastapi uvicorn pyngrok
!pip install sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 3.1 MB/s eta 0:00:00


In [2]:
# ngrokでトンネリング
from pyngrok import ngrok

import torch
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
from pydantic import BaseModel
import uvicorn
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
# 10分くらいかかります
# Hugging Face からモデルロード
# 日本語Llama ELYZAモデル例: "elyza/Llama-3-ELYZA-JP-8B"
model_name = "elyza/Llama-3-ELYZA-JP-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
)
model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (n

In [3]:
# systemメッセージ（モデルキャラクター・行動方針）
DEFAULT_SYSTEM_PROMPT = "あなたは誠実で優秀な日本人のアシスタントです。特に指示が無い場合は、常に日本語で回答してください。"

In [4]:
# Chat形式プロンプト生成用関数
def build_prompt(user_message: str) -> str:
    messages = [
        {"role": "system", "content": DEFAULT_SYSTEM_PROMPT},
        {"role": "user", "content": user_message}
    ]
    # `apply_chat_template`はELYZAモデル付属のtransformers_llama_japanese拡張で利用可能な仮想メソッド例
    # 必要に応じて、適宜 `apply_chat_template` に相当する独自実装またはモデルに対応したフォーマット生成を行う
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt

In [5]:
# 推論（応答生成）関数
def generate_response(user_message: str) -> str:
    prompt = build_prompt(user_message)
    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

    with torch.no_grad():
        output_ids = model.generate(
            token_ids.to(model.device),
            max_new_tokens=1024,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
        )
    output = tokenizer.decode(output_ids[0][token_ids.size(1):], skip_special_tokens=True)
    return output.strip()

def generate_response_dummy(user_message: str) -> str:
    """ テスト用（CPUインスタンスで実行可能なように） """
    return "dummy: こんにちは！お元気ですか？"

In [6]:
# FastAPI定義
app = FastAPI()

# CORS設定
app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

# スキーマ定義
class RequestBody(BaseModel):
    message: str

@app.post("/predict")
async def predict(req: RequestBody):
    # response_text = generate_response(req.message)
    response_text = generate_response_dummy(req.message)
    return {"response": response_text}


In [7]:
# ngrokでAPI URLを公開する
import os
from google.colab import userdata
my_token = userdata.get("NGROK_TOKEN")

ngrok.set_auth_token(my_token)

ngrok_tunnel = ngrok.connect(8000)
print('PUBLIC_URL:',ngrok_tunnel.public_url)

nest_asyncio.apply()

uvicorn.run(app,port=8000)
# サーバ起動中はセルが動作し続けます

PUBLIC_URL: https://883c-35-230-168-139.ngrok-free.app


INFO:     Started server process [482]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     54.86.50.139:0 - "POST /predict HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [482]


In [ ]:
# （ランタイム手動切断用）
from google.colab import runtime
runtime.unassign()